In [30]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [31]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [32]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [33]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [34]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
bnf_1min = pd.read_csv("../data/bnf.csv")
# bnf_1min = pd.read_csv('../data/nifty_min.csv')
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2017) & (bnf_1min["datetime"].dt.year <= 2024)
]

In [35]:
# # If Stocks Data ...
# bnf_pandas['datetime'] = pd.to_datetime(bnf_pandas['time'])
# bnf_pandas['datetime'] = bnf_pandas['datetime'].dt.tz_localize(None)
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2017]
# bnf_pandas.drop(columns=['time'], inplace=True)
# # bnf_pandas

In [36]:
# bnf = pl.DataFrame(bnf_pandas)
# bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
# bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# # bnf

In [37]:
# bnf_1min

In [38]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [40]:
def calculate_weekly_ranges(df):
    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)
    # Resample to weekly data to get the high and low for each week
    weekly_ranges = df.resample("W-FRI").agg({"high": "max", "low": "min"})
    return weekly_ranges


async def find_breakouts(df, weekly_ranges):

    results = []
    multiplier_to_range = 0.5

    portfolio_value = 10_00_000
    index_leverage = 7

    index_name = "BANKNIFTY"
    index_str_for_opt = "bnf"

    weekly_ranges["datetime"] = pd.to_datetime(weekly_ranges["datetime"])
    weekly_ranges = weekly_ranges[:-1]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    for i in range(1, len(weekly_ranges)):
        previous_week = weekly_ranges.iloc[i - 1]
        current_week = weekly_ranges.iloc[i]
        print("Current Week : ", current_week["datetime"].date())
        if current_week["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_week_high = weekly_ranges.iloc[i - 1]["high"]
        previous_week_low = weekly_ranges.iloc[i - 1]["low"]
        previous_week_close = weekly_ranges.iloc[i - 1]["close"]

        current_week_open = weekly_ranges.iloc[i]["open"]

        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_week_data = df.loc[
            (df["datetime"] >= current_week["datetime"])
            & (df["datetime"] <= (current_week["datetime"] + pd.Timedelta(days=7)))
        ]
        # print("Current Week Data:\n", current_week_data)

        weekly_range = previous_week_high - previous_week_low
        addition_range = multiplier_to_range * weekly_range

        # Check for breakouts
        high_level = current_week_data["open"].iloc[0] + addition_range
        low_level = current_week_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_week_data[current_week_data["high"] >= high_level]
        breakout_low = current_week_data[current_week_data["low"] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 100) * 100)
            # atm_strike = int(round(((high_level+low_level) / 2) / 100) * 100)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(
                current_week["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 30),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None :
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_high_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            low_breach = current_week_data_after_entry[
                current_week_data_after_entry["low"] < previous_week_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    exit_price = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c",
                    ].iloc[0]

                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Friday Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 100) * 100)
            # atm_strike = int(round(((high_level+low_level) / 2) / 100) * 100)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(
                current_week["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 30),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_low_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            high_breach = current_week_data_after_entry[
                current_week_data_after_entry["high"] > previous_week_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    exit_price = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c",
                    ].iloc[0]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results


async def trade():
    df = bnf_1min
    # weekly_ranges = calculate_weekly_ranges(df)
    print(df.head())
    df =df.rename(
        {"open": "o", "high": "h", "low": "l", "close": "c"}
    )
    print(df.head())
    # df = df.select(["datetime", "o", "h", "l", "c"])
    weekly_ranges = resample(pl.DataFrame(df), "7d", pd.Timedelta(days=1))
    weekly_ranges = weekly_ranges.to_pandas()

    breakouts = await find_breakouts(df, weekly_ranges)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas


tradebook = await trade()
# tradebook

             datetime       open       high        low      close  volume
0 2017-01-02 09:15:00 18242.3000 18248.2000 18175.9000 18181.2000       0
1 2017-01-02 09:16:00 18181.8500 18194.7000 18179.9500 18184.4500       0
2 2017-01-02 09:17:00 18184.9500 18189.2500 18133.8000 18133.8000       0
3 2017-01-02 09:18:00 18135.1000 18141.5500 18118.5500 18138.9500       0
4 2017-01-02 09:19:00 18138.9500 18142.5500 18120.4500 18124.3000       0
             datetime       open       high        low      close  volume
0 2017-01-02 09:15:00 18242.3000 18248.2000 18175.9000 18181.2000       0
1 2017-01-02 09:16:00 18181.8500 18194.7000 18179.9500 18184.4500       0
2 2017-01-02 09:17:00 18184.9500 18189.2500 18133.8000 18133.8000       0
3 2017-01-02 09:18:00 18135.1000 18141.5500 18118.5500 18138.9500       0
4 2017-01-02 09:19:00 18138.9500 18142.5500 18120.4500 18124.3000       0
Current Week :  2017-01-06
Current Week :  2017-01-13
Current Week :  2017-01-20
Current Week :  2017-01-27
Curr

In [41]:
tradebook

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs
0,2016-12-30,18248.2000,17831.7500,416.4500,18168.4500,50%,18376.6750,17960.2250,high,18000,2017-01-12,2,BANKNIFTY1711218000PE,PE,2017-01-10 09:17:00,15.9000,2017-01-12 15:30:00,0.0500,Friday Closing,15.8500,0.1595,15.6905,388.8889,6163.8889,6101.8611,0.6164,0.6102
1,2017-01-13,19276.2500,18781.6000,494.6500,18996.6000,50%,19243.9250,18749.2750,high,18700,2017-01-25,0,BANKNIFTY17JAN18700PE,PE,2017-01-25 13:05:00,0.3000,2017-01-25 15:30:00,0.0500,Friday Closing,0.2500,0.0035,0.2465,374.3316,93.5829,92.2727,0.0094,0.0092
2,2017-01-13,19276.2500,18781.6000,494.6500,18996.6000,50%,19243.9250,18749.2750,low,19200,2017-01-25,2,BANKNIFTY17JAN19200CE,CE,2017-01-23 09:15:00,15.5000,2017-01-25 13:38:00,58.4500,SL Hit,-42.9500,0.7395,-43.6895,364.5833,-15658.8542,-15928.4635,-1.5659,-1.5928
3,2017-01-20,19517.7000,18725.0500,792.6500,19588.4000,50%,19984.7250,19192.0750,high,19200,2017-02-02,1,BANKNIFTY1720219200PE,PE,2017-02-01 14:53:00,1.9500,2017-02-02 15:30:00,0.1000,Friday Closing,1.8500,0.0205,1.8295,364.5833,674.4792,667.0052,0.0674,0.0667
4,2017-01-27,20145.9000,19435.7500,710.1500,20061.9000,50%,20416.9750,19706.8250,high,19700,2017-02-09,3,BANKNIFTY1720919700PE,PE,2017-02-06 09:24:00,10.7500,2017-02-09 15:30:00,0.0500,Friday Closing,10.7000,0.1080,10.5920,355.3299,3802.0305,3763.6548,0.3802,0.3764
5,2017-02-10,20366.7000,20088.4000,278.3000,20918.4500,50%,21057.6000,20779.3000,low,21100,2017-02-23,6,BANKNIFTY17FEB21100CE,CE,2017-02-17 09:40:00,17.9500,2017-02-17 09:41:00,15.6000,SL Hit,2.3500,0.3355,2.0145,331.7536,779.6209,668.3175,0.0780,0.0668
6,2017-02-17,21031.6500,20452.6000,579.0500,20853.4500,50%,21142.9750,20563.9250,low,21100,2017-03-02,2,BANKNIFTY1730221100CE,CE,2017-02-28 11:42:00,6.8000,2017-03-02 15:30:00,0.0500,Expiry Closing,6.7500,0.0685,6.6815,331.7536,2239.3365,2216.6114,0.2239,0.2217
7,2017-02-24,20901.0000,20518.5000,382.5000,20572.8500,50%,20764.1000,20381.6000,high,20400,2017-03-09,0,BANKNIFTY1730920400PE,PE,2017-03-09 14:26:00,0.2000,2017-03-09 15:30:00,0.0500,Friday Closing,0.1500,0.0025,0.1475,343.1373,51.4706,50.6127,0.0051,0.0051
8,2017-03-03,20777.1500,20423.7000,353.4500,20825.9000,50%,21002.6250,20649.1750,high,20600,2017-03-16,2,BANKNIFTY1731620600PE,PE,2017-03-14 09:15:00,12.0000,2017-03-16 15:30:00,0.0500,Friday Closing,11.9500,0.1205,11.8295,339.8058,4060.6796,4019.7330,0.4061,0.4020
9,2017-03-10,21289.3000,20653.5000,635.8000,21308.4500,50%,21626.3500,20990.5500,low,21600,2017-03-23,2,BANKNIFTY1732321600CE,CE,2017-03-21 12:32:00,1.5000,2017-03-23 15:30:00,0.0500,Expiry Closing,1.4500,0.0155,1.4345,324.0741,469.9074,464.8843,0.0470,0.0465


In [42]:
# positive = tradebook[tradebook['Points']<0]
# positive['Points'].sum()
tradebook["ROI% w cs"].sum()
# no_value = tradebook[(tradebook['ROI%'] > 0) | (tradebook['ROI%'] < 0)]
# len(no_value)

166.6100883924972

In [43]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
tradebook["Trade Year"] = tradebook["Entry Time"].dt.year

In [44]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w cs"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0][
    "ROI% w cs"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0][
    "ROI% w cs"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w cs"].cumsum()
    - combined_df_sorted["ROI% w cs"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,5.8936,46,78.2609,0.3178,-0.6932,-1.9699,2.9918
2018,9.7558,47,82.9787,0.5515,-1.4690,-4.3741,2.2304
2019,23.2792,44,84.0909,0.8230,-1.1955,-2.7869,8.3529
2020,54.5446,46,95.6522,1.2492,-0.2107,-0.2185,249.5900
2021,33.3346,38,94.7368,1.0624,-2.4565,-2.8614,11.6499
2022,30.6443,40,97.5000,0.7942,-0.3307,-0.3307,92.6611
2023,8.4546,41,82.9268,0.4803,-1.5754,-4.4754,1.8891
2024,0.7033,24,33.3333,0.3267,-1.9099,-1.9099,0.3683
Overall,166.6101,326.0000,83.7423,0.7565,-1.2098,-4.4754,37.2277


In [34]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w cs"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0][
    "ROI% w cs"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0][
    "ROI% w cs"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w cs"].cumsum()
    - combined_df_sorted["ROI% w cs"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,5.8936,46,78.2609,0.3178,-0.6932,-1.9699,2.9918
2018,9.7558,47,82.9787,0.5515,-1.4690,-4.3741,2.2304
2019,23.2792,44,84.0909,0.8230,-1.1955,-2.7869,8.3529
2020,54.5446,46,95.6522,1.2492,-0.2107,-0.2185,249.5900
2021,33.3346,38,94.7368,1.0624,-2.4565,-2.8614,11.6499
2022,30.6443,40,97.5000,0.7942,-0.3307,-0.3307,92.6611
2023,8.4546,41,82.9268,0.4803,-1.5754,-4.4754,1.8891
2024,1.7789,13,76.9231,0.3689,-1.9099,-1.9099,0.9314
Overall,167.6856,315.0000,87.3016,0.7549,-1.2098,-4.4754,37.4681


50% MOVE VARIATION FROM CURRENT OPEN, ENTRY AT OPP LEVELS, SL AT PRV. WK H/L

In [39]:
# stats_of_trades = tradebook
# stats_of_trades['Cumulative ROI%'] = stats_of_trades.groupby('Option Type')['ROI%'].cumsum()
# stats_of_trades['Running Max ROI%'] = stats_of_trades.groupby('Option Type')['Cumulative ROI%'].cummax()
# stats_of_trades['Drawdown'] = stats_of_trades['Cumulative ROI%'] - stats_of_trades['Running Max ROI%']
# max_dd_distribution = stats_of_trades.groupby('Option Type')['Drawdown'].min().reset_index()
# max_dd_distribution.rename(columns={'Drawdown': 'Max Drawdown'}, inplace=True)
# roi_distribution = stats_of_trades.groupby('Option Type')['ROI%'].sum().reset_index()
# grouped = pd.merge(roi_distribution, max_dd_distribution, on='Option Type')
# grouped['ROI/DD Ratio'] = grouped['ROI%'] / grouped['Max Drawdown'].abs()
# grouped

In [45]:
tradebook.head()

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs,Trade Year
0,2016-12-30,18248.2000,17831.7500,416.4500,18168.4500,50%,18376.6750,17960.2250,high,18000,2017-01-12,2,BANKNIFTY1711218000PE,PE,2017-01-10 09:17:00,15.9000,2017-01-12 15:30:00,0.0500,Friday Closing,15.8500,0.1595,15.6905,388.8889,6163.8889,6101.8611,0.6164,0.6102,2017
1,2017-01-13,19276.2500,18781.6000,494.6500,18996.6000,50%,19243.9250,18749.2750,high,18700,2017-01-25,0,BANKNIFTY17JAN18700PE,PE,2017-01-25 13:05:00,0.3000,2017-01-25 15:30:00,0.0500,Friday Closing,0.2500,0.0035,0.2465,374.3316,93.5829,92.2727,0.0094,0.0092,2017
2,2017-01-13,19276.2500,18781.6000,494.6500,18996.6000,50%,19243.9250,18749.2750,low,19200,2017-01-25,2,BANKNIFTY17JAN19200CE,CE,2017-01-23 09:15:00,15.5000,2017-01-25 13:38:00,58.4500,SL Hit,-42.9500,0.7395,-43.6895,364.5833,-15658.8542,-15928.4635,-1.5659,-1.5928,2017
3,2017-01-20,19517.7000,18725.0500,792.6500,19588.4000,50%,19984.7250,19192.0750,high,19200,2017-02-02,1,BANKNIFTY1720219200PE,PE,2017-02-01 14:53:00,1.9500,2017-02-02 15:30:00,0.1000,Friday Closing,1.8500,0.0205,1.8295,364.5833,674.4792,667.0052,0.0674,0.0667,2017
4,2017-01-27,20145.9000,19435.7500,710.1500,20061.9000,50%,20416.9750,19706.8250,high,19700,2017-02-09,3,BANKNIFTY1720919700PE,PE,2017-02-06 09:24:00,10.7500,2017-02-09 15:30:00,0.0500,Friday Closing,10.7000,0.1080,10.5920,355.3299,3802.0305,3763.6548,0.3802,0.3764,2017


In [48]:
def round_to_nearest_100(value):
    return 100 * round(value / 100)


async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    hedge_pct = 3
    index_name = "BANKNIFTY"
    index_str_for_opt = "bnf"

    # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    df["Hedge Strike"] = df.apply(
        lambda row: (
            round_to_nearest_100(row["Strike"] * (1 + hedge_pct / 100))
            if row["Option Type"] == "CE"
            else round_to_nearest_100(row["Strike"] * (1 - hedge_pct / 100))
        ),
        axis=1,
    )

    df["Hedge Contract"] = df.apply(
        lambda row: get_option_contract_name2(
            index_name, row["Hedge Strike"], row["Expiry"], row["Option Type"]
        ),
        axis=1,
    )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    for i in range(0, len(df)):
        print(df.iloc[i]["Week"])
        hedge_strike = int(df.iloc[i]["Hedge Strike"])
        hedge_expiry = df.iloc[i]["Expiry"]
        hedge_opt_type = df.iloc[i]["Option Type"][-2:-1]
        hedge_entry_datetime = df.iloc[i]["Entry Time"]
        hedge_exit_datetime = df.iloc[i]["Exit Time"]

        hedge_df = await fetch_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=hedge_strike,
            asset_class=hedge_opt_type,
        )
        if not isinstance(hedge_df, str) and hedge_df is not None:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            df.loc[i, "Hedge Entry Price"] = 0
            df.loc[i, "Hedge Exit Price"] = 0
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]["c"]
        hedge_exit_price = hedge_df.iloc[-1]["c"]

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, "Hedge Entry Price"] = hedge_entry_price
        df.loc[i, "Hedge Exit Price"] = hedge_exit_price

    return df


tb_with_hedge_tmp = await add_hedges(tradebook)
# tb_with_hedge

2016-12-30
1.0 0.05
2017-01-13
0.1 0.05
2017-01-13
2.05 0.1
2017-01-20
0.8 0.05
2017-01-27
1.75 0.1
2017-02-10
None
2017-02-17
0.5 0.05
2017-02-24
0.05 0.05
2017-03-03
16.35 0.05
2017-03-10
0.45 0.05
2017-03-17
1.55 0.05
2017-03-31
1.4 0.05
2017-04-07
1.4 0.05
2017-04-07
0.25 0.05
2017-04-14
2.95 0.05
2017-04-28
None
2017-05-05
0.5 0.15
2017-05-12
0.2 0.05
2017-05-12
0.9 0.05
2017-05-26
0.1 0.05
2017-06-02
3.75 0.05
2017-06-09
1.9 0.05
2017-06-16
2.35 0.05
2017-06-23
0.1 0.05
2017-06-30
2.55 0.05
2017-07-07
0.05 0.05
2017-07-14
2.85 0.1
2017-07-28
1.25 2.5
2017-07-28
0.75 0.05
2017-08-04
0.65 0.05
2017-08-25
1.35 0.05
2017-09-01
2.5 0.05
2017-09-15
2.2 0.05
2017-09-29
4.15 0.3
2017-10-06
None
2017-10-06
None
2017-10-13
0.65 0.05
2017-10-20
0.7 0.05
2017-10-27
0.4 0.05
2017-11-03
3.5 0.05
2017-11-10
None
2017-11-17
3.6 0.5
2017-11-17
0.1 0.05
2017-11-24
1.35 0.05
2017-12-08
0.6 0.05
2017-12-08
5.7 6.8
2017-12-22
0.8 0.05
2017-12-29
1.2 0.05
2018-01-05
3.5 0.05
2018-01-05
0.95 1.8
2018-0

In [49]:
# tb_with_hedge = pd.read_csv('JJMS TB w Hedges 1%.csv')
# tb_with_hedge

In [50]:
tb_with_hedge = tb_with_hedge_tmp
tb_with_hedge

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs,Trade Year,Hedge Strike,Hedge Contract,Hedge Entry Price,Hedge Exit Price
0,2016-12-30,18248.2000,17831.7500,416.4500,18168.4500,50%,18376.6750,17960.2250,high,18000,2017-01-12,2,BANKNIFTY1711218000PE,PE,2017-01-10 09:17:00,15.9000,2017-01-12 15:30:00,0.0500,Friday Closing,15.8500,0.1595,15.6905,388.8889,6163.8889,6101.8611,0.6164,0.6102,2017,17500,BANKNIFTY1711217500PE,1.0000,0.0500
1,2017-01-13,19276.2500,18781.6000,494.6500,18996.6000,50%,19243.9250,18749.2750,high,18700,2017-01-25,0,BANKNIFTY17JAN18700PE,PE,2017-01-25 13:05:00,0.3000,2017-01-25 15:30:00,0.0500,Friday Closing,0.2500,0.0035,0.2465,374.3316,93.5829,92.2727,0.0094,0.0092,2017,18100,BANKNIFTY17JAN18100PE,0.1000,0.0500
2,2017-01-13,19276.2500,18781.6000,494.6500,18996.6000,50%,19243.9250,18749.2750,low,19200,2017-01-25,2,BANKNIFTY17JAN19200CE,CE,2017-01-23 09:15:00,15.5000,2017-01-25 13:38:00,58.4500,SL Hit,-42.9500,0.7395,-43.6895,364.5833,-15658.8542,-15928.4635,-1.5659,-1.5928,2017,19800,BANKNIFTY17JAN19800CE,2.0500,0.1000
3,2017-01-20,19517.7000,18725.0500,792.6500,19588.4000,50%,19984.7250,19192.0750,high,19200,2017-02-02,1,BANKNIFTY1720219200PE,PE,2017-02-01 14:53:00,1.9500,2017-02-02 15:30:00,0.1000,Friday Closing,1.8500,0.0205,1.8295,364.5833,674.4792,667.0052,0.0674,0.0667,2017,18600,BANKNIFTY1720218600PE,0.8000,0.0500
4,2017-01-27,20145.9000,19435.7500,710.1500,20061.9000,50%,20416.9750,19706.8250,high,19700,2017-02-09,3,BANKNIFTY1720919700PE,PE,2017-02-06 09:24:00,10.7500,2017-02-09 15:30:00,0.0500,Friday Closing,10.7000,0.1080,10.5920,355.3299,3802.0305,3763.6548,0.3802,0.3764,2017,19100,BANKNIFTY1720919100PE,1.7500,0.1000
5,2017-02-10,20366.7000,20088.4000,278.3000,20918.4500,50%,21057.6000,20779.3000,low,21100,2017-02-23,6,BANKNIFTY17FEB21100CE,CE,2017-02-17 09:40:00,17.9500,2017-02-17 09:41:00,15.6000,SL Hit,2.3500,0.3355,2.0145,331.7536,779.6209,668.3175,0.0780,0.0668,2017,21700,BANKNIFTY17FEB21700CE,0.0000,0.0000
6,2017-02-17,21031.6500,20452.6000,579.0500,20853.4500,50%,21142.9750,20563.9250,low,21100,2017-03-02,2,BANKNIFTY1730221100CE,CE,2017-02-28 11:42:00,6.8000,2017-03-02 15:30:00,0.0500,Expiry Closing,6.7500,0.0685,6.6815,331.7536,2239.3365,2216.6114,0.2239,0.2217,2017,21700,BANKNIFTY1730221700CE,0.5000,0.0500
7,2017-02-24,20901.0000,20518.5000,382.5000,20572.8500,50%,20764.1000,20381.6000,high,20400,2017-03-09,0,BANKNIFTY1730920400PE,PE,2017-03-09 14:26:00,0.2000,2017-03-09 15:30:00,0.0500,Friday Closing,0.1500,0.0025,0.1475,343.1373,51.4706,50.6127,0.0051,0.0051,2017,19800,BANKNIFTY1730919800PE,0.0500,0.0500
8,2017-03-03,20777.1500,20423.7000,353.4500,20825.9000,50%,21002.6250,20649.1750,high,20600,2017-03-16,2,BANKNIFTY1731620600PE,PE,2017-03-14 09:15:00,12.0000,2017-03-16 15:30:00,0.0500,Friday Closing,11.9500,0.1205,11.8295,339.8058,4060.6796,4019.7330,0.4061,0.4020,2017,20000,BANKNIFTY1731620000PE,16.3500,0.0500
9,2017-03-10,21289.3000,20653.5000,635.8000,21308.4500,50%,21626.3500,20990.5500,low,21600,2017-03-23,2,BANKNIFTY1732321600CE,CE,2017-03-21 12:32:00,1.5000,2017-03-23 15:30:00,0.0500,Expiry Closing,1.4500,0.0155,1.4345,324.0741,469.9074,464.8843,0.0470,0.0465,2017,22200,BANKNIFTY1732322200CE,0.4500,0.0500


In [51]:
tb_with_hedge['Qty'] = tb_with_hedge['Qty'] * 2.5
tb_with_hedge["Hedge Points"] = (
    tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
)
tb_with_hedge["PnL"] = tb_with_hedge["Points"] * tb_with_hedge["Qty"]
tb_with_hedge["PnL w cs"] = tb_with_hedge["Points w cs"] * tb_with_hedge["Qty"]
tb_with_hedge["Hedge PnL"] = tb_with_hedge["Hedge Points"] * tb_with_hedge["Qty"]
tb_with_hedge["Total PnL"] = tb_with_hedge["PnL w cs"] + tb_with_hedge["Hedge PnL"]

In [52]:
tb_with_hedge["Total ROI%"] = tb_with_hedge["Total PnL"] * 100 / 1000000
tb_with_hedge["Entry Time"] = pd.to_datetime(tb_with_hedge["Entry Time"])
tb_with_hedge["Trade Year"] = tb_with_hedge["Entry Time"].dt.year

In [53]:
# stats_of_trades = tb_with_hedge
# stats_of_trades["Cumulative ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Total ROI%"
# ].cumsum()
# stats_of_trades["Running Max ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Cumulative ROI%"
# ].cummax()
# stats_of_trades["Drawdown"] = (
#     stats_of_trades["Cumulative ROI%"] - stats_of_trades["Running Max ROI%"]
# )
# max_dd_distribution = (
#     stats_of_trades.groupby("Option Type")["Drawdown"].min().reset_index()
# )
# max_dd_distribution.rename(columns={"Drawdown": "Max Drawdown"}, inplace=True)
# roi_distribution = (
#     stats_of_trades.groupby("Option Type")["Total ROI%"].sum().reset_index()
# )
# grouped = pd.merge(roi_distribution, max_dd_distribution, on="Option Type")
# grouped["ROI/DD Ratio"] = grouped["Total ROI%"] / grouped["Max Drawdown"].abs()
# grouped

In [54]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,9.7753,46,73.9130,0.7174,-1.4615,-5.0608,1.9316
2018,16.0717,47,82.9787,1.1796,-3.7415,-11.8167,1.3601
2019,37.1448,44,84.0909,1.4738,-2.8975,-6.4003,5.8036
2020,86.4705,46,91.3043,2.0866,-0.2918,-0.6333,136.5436
2021,54.1443,38,94.7368,1.8590,-6.3899,-9.4434,5.7336
2022,58.4285,40,97.5000,1.5199,-0.8486,-0.8486,68.8520
2023,12.4177,41,82.9268,0.9281,-3.8276,-11.2883,1.1000
2024,0.9178,24,33.3333,0.6679,-4.4257,-4.4257,0.2074
Overall,275.3707,326.0000,82.5153,1.3965,-2.7106,-11.8167,23.3035


In [51]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,9.7753,46,73.9130,0.7174,-1.4615,-5.0608,1.9316
2018,16.0717,47,82.9787,1.1796,-3.7415,-11.8167,1.3601
2019,37.1448,44,84.0909,1.4738,-2.8975,-6.4003,5.8036
2020,86.4705,46,91.3043,2.0866,-0.2918,-0.6333,136.5436
2021,54.1443,38,94.7368,1.8590,-6.3899,-9.4434,5.7336
2022,58.4285,40,97.5000,1.5199,-0.8486,-0.8486,68.8520
2023,12.4177,41,82.9268,0.9281,-3.8276,-11.2883,1.1000
2024,3.0138,13,76.9231,0.7440,-4.4257,-4.4257,0.6810
Overall,277.4667,315.0000,86.0317,1.3939,-2.7106,-11.8167,23.4809


BNF Stats With 4pct Away hedges, qty = 2x RBOS

In [55]:
a = tb_with_hedge["PnL w cs"].sum()
b = tb_with_hedge["Hedge PnL"].sum()
c = tb_with_hedge["Total PnL"].sum()
d = tb_with_hedge["Total ROI%"].sum()
print(a, b, c, d)

4165252.20981243 -1411545.7054142058 2753706.5043982244 275.37065043982244


In [56]:
# tb_with_hedge.to_csv('JJMS TB w Hedges 1%.csv')

In [57]:
# # tb_with_hedge['DD%'] = (tb_with_hedge['Total ROI%'].cumsum() - tb_with_hedge['Total ROI%'].cumsum().cummax())
# tb_with_hedge.drop(columns=['Cumulative ROI%', 'Running Max ROI%', 'Drawdown'], inplace=True)
# tb_with_hedge

In [58]:
# tb_with_hedge["ROI%"] = tb_with_hedge["PnL"] * 100 / 1000000

In [59]:
tb_with_hedge.to_csv('BNF TB RBOS w hedge 3pct_2pt5x.csv')

In [24]:
stats_df9 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df9.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df9["Total ROI"].sum()
overall_total_trades = stats_df9["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df9.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df9

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,9.0137,46,69.5652,0.8365,-1.7754,-5.1809,1.7398
2018,13.7351,47,82.9787,1.2615,-4.4330,-14.5574,0.9435
2019,37.4229,44,81.8182,1.5129,-2.4346,-6.4494,5.8026
2020,81.2101,46,91.3043,1.9495,-0.2230,-0.6443,126.0353
2021,50.0985,38,94.7368,1.7970,-7.2968,-11.1895,4.4773
2022,58.3171,40,97.5000,1.5206,-0.9847,-0.9847,59.2230
2023,11.3569,41,80.4878,0.9956,-4.2995,-13.0583,0.8697
2024,3.6898,13,76.9231,0.7755,-4.0648,-4.0648,0.9077
Overall,264.8440,315.0000,86.9841,1.4117,-3.0289,-14.5574,18.1931


In [42]:
tradebook["Cumulative ROI%"] = tradebook["ROI% w cs"].cumsum()
tradebook["Running Max ROI%"] = tradebook["Cumulative ROI%"].cummax()
tradebook["Drawdown"] = tradebook["Cumulative ROI%"] - tradebook["Running Max ROI%"]
tradebook

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Qty,PnL,ROI%,Trade Year,Slippage in pts,PnL w cs,ROI% w cs,Cumulative ROI%,Running Max ROI%,Drawdown
0,2019-12-27,32463.1000,32058.5500,404.5500,32326.9500,50%,32529.2250,32124.6750,low,32500,2020-01-09,6,BANKNIFTY2010932500CE,CE,2020-01-03 11:03:00,99.9500,2020-01-08 15:20:00,0.0500,Expiry Closing,99.9000,199.8203,19962.0495,1.9962,2020,1.0000,19762.2292,1.9762,1.9762,1.9762,0.0000
1,2020-01-10,32345.2000,31671.5000,673.7000,31571.6500,50%,31908.5000,31234.8000,high,31200,2020-01-23,3,BANKNIFTY2012331200PE,PE,2020-01-20 09:15:00,70.4000,2020-01-20 09:21:00,78.6500,SL Hit,-8.2500,203.7075,-1680.5867,-0.1681,2020,1.4905,-1984.2127,-0.1984,1.7778,1.9762,-0.1984
2,2020-01-10,32345.2000,31671.5000,673.7000,31571.6500,50%,31908.5000,31234.8000,low,31900,2020-01-23,3,BANKNIFTY2012331900CE,CE,2020-01-20 13:00:00,63.9000,2020-01-22 15:20:00,0.0500,Expiry Closing,63.8500,203.7075,13006.7223,1.3007,2020,0.6395,12876.4514,1.2876,3.0654,3.0654,0.0000
3,2020-01-24,31374.1500,30522.0500,852.1000,30950.5000,50%,31376.5500,30524.4500,low,31400,2020-02-06,5,BANKNIFTY2020631400CE,CE,2020-02-01 09:15:00,228.9500,2020-02-05 15:20:00,0.1500,Expiry Closing,228.8000,207.1611,47398.4552,4.7398,2020,2.2910,46923.8492,4.6924,7.7578,7.7578,0.0000
4,2020-02-07,31598.5500,30958.3500,640.2000,31281.8500,50%,31601.9500,30961.7500,low,31600,2020-02-20,6,BANKNIFTY2022031600CE,CE,2020-02-14 10:55:00,44.0000,2020-02-19 15:20:00,0.0500,Expiry Closing,43.9500,205.6835,9039.7903,0.9040,2020,0.4405,8949.1867,0.8949,8.6528,8.6528,0.0000
5,2020-02-21,30702.1500,29918.8500,783.3000,29501.5500,50%,29893.2000,29109.9000,low,29900,2020-03-05,6,BANKNIFTY2030529900CE,CE,2020-02-28 15:00:00,129.2000,2020-03-04 15:20:00,0.1000,Expiry Closing,129.1000,217.4408,28071.6016,2.8072,2020,1.2930,27790.4507,2.7790,11.4318,11.4318,0.0000
6,2020-02-28,29778.6500,28125.4500,1653.2000,27649.9500,50%,28476.5500,26823.3500,low,28500,2020-03-12,3,BANKNIFTY2031228500CE,CE,2020-03-09 09:37:00,33.1000,2020-03-11 15:20:00,0.2000,Expiry Closing,32.9000,228.2580,7509.6878,0.7510,2020,0.3330,7433.6779,0.7434,12.1752,12.1752,0.0000
7,2020-03-06,28042.5000,23613.1000,4429.4000,22720.7500,50%,24935.4500,20506.0500,high,20500,2020-03-19,6,BANKNIFTY2031920500PE,PE,2020-03-13 12:37:00,297.0500,2020-03-16 09:24:00,297.0500,SL Hit,0.0000,260.6731,0.0000,0.0000,2020,5.9410,-1548.6586,-0.1549,12.0203,12.1752,-0.1549
8,2020-03-06,28042.5000,23613.1000,4429.4000,22720.7500,50%,24935.4500,20506.0500,low,24900,2020-03-19,1,BANKNIFTY2031924900CE,CE,2020-03-18 12:32:00,9.0000,2020-03-18 15:20:00,0.1000,Expiry Closing,8.9000,260.6731,2319.9902,0.2320,2020,0.0910,2296.2690,0.2296,12.2499,12.2499,0.0000
9,2020-03-13,25654.9500,18680.6500,6974.3000,19898.3000,50%,23385.4500,16411.1500,low,23400,2020-03-26,2,BANKNIFTY20MAR23400CE,CE,2020-03-24 09:52:00,34.0500,2020-03-25 15:20:00,1.0000,Expiry Closing,33.0500,277.9506,9186.2675,0.9186,2020,0.3505,9088.8458,0.9089,13.1588,13.1588,0.0000


In [43]:
tradebook.to_csv("2020 Trades JJMS OS.csv")